In [1]:
import os
import time
import torch
import torchvision.transforms as transforms
from torchvision import datasets
from torchvision.models import vit_b_16, ViT_B_16_Weights
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import numpy as np



In [2]:
# GPU ni tekshirish
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"GPU mavjud: {torch.cuda.get_device_name(device)}")
else:
    device = torch.device("cpu")
    print("GPU mavjud emas, CPU ishlatiladi.")

torch.cuda.empty_cache()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

GPU mavjud: NVIDIA GeForce RTX 3060
cuda


In [3]:
# Ma'lumotlar to'plamini yuklash va transformatsiyalar
data_dir = 'face_emoc_ds'
batch_size = 32
image_size = 224

# Data augmentation va transformatsiyalar
transform = transforms.Compose([
    transforms.Lambda(lambda img: img if img.mode == 'RGB' else img.convert('RGB')),
    transforms.Resize((image_size, image_size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomResizedCrop(image_size, scale=(0.8, 1.0)),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.RandomPerspective(distortion_scale=0.2, p=0.5),
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.7, scale=(0.02, 0.2), ratio=(0.3, 3.3), value=(0, 0, 0)),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = datasets.ImageFolder(data_dir, transform=transform)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

class_names = dataset.classes
print("Class names:", class_names)

# Class Weights hisoblash
class_counts = np.array([9390, 5002, 9405, 9851, 9554, 9722, 10093, 9395])
num_samples = np.sum(class_counts)
class_weights = num_samples / (len(class_counts) * class_counts)
class_weights = torch.FloatTensor(class_weights).to(device)
print("Class Weights:", class_weights)

Class names: ['anger', 'contempt', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
Class Weights: tensor([0.9640, 1.8096, 0.9624, 0.9188, 0.9474, 0.9310, 0.8968, 0.9634],
       device='cuda:0')


In [4]:
# Modelni yuklash va sozlash
def load_model(model_name, num_classes):
    if model_name == 'vit_b_16':
        model = vit_b_16(weights=ViT_B_16_Weights.DEFAULT)
        model.heads.head = nn.Linear(model.heads.head.in_features, num_classes)
    else:
        raise ValueError("Only 'vit_b_16' is supported.")
    for param in model.parameters():
        param.requires_grad = True
    return model.to(device)

model_name = 'vit_b_16'
model = load_model(model_name, len(class_names))

In [5]:
# Loss va optimizer
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)

c:\Program Files\Python312\Lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [6]:
# Trening va validatsiya funksiyasi
def train_and_validate(model, criterion, optimizer, train_loader, val_loader, num_epochs=10):
    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []
    epoch_times = []

    with open("training_results.txt", "w") as f:
        f.write("Epoch, Train Loss, Train Accuracy, Val Loss, Val Accuracy, Time (sec)\n")

    for epoch in range(num_epochs):
        start_time = time.time()

        # O'quv rejimi
        model.train()
        train_loss, train_correct = 0.0, 0

        for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            train_correct += torch.sum(preds == labels.data)

        epoch_train_loss = train_loss / len(train_loader.dataset)
        epoch_train_acc = 100 * train_correct.double() / len(train_loader.dataset)
        train_losses.append(epoch_train_loss)
        train_accuracies.append(epoch_train_acc)

        # Validatsiya rejimi
        model.eval()
        val_loss, val_correct = 0.0, 0

        with torch.no_grad():
            for inputs, labels in tqdm(val_loader, desc="Validation"):
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                val_correct += torch.sum(preds == labels.data)

        epoch_val_loss = val_loss / len(val_loader.dataset)
        epoch_val_acc = 100 * val_correct.double() / len(val_loader.dataset)
        val_losses.append(epoch_val_loss)
        val_accuracies.append(epoch_val_acc)

        epoch_time = time.time() - start_time
        epoch_times.append(epoch_time)

        print(f'Epoch {epoch+1}/{num_epochs}, '
              f'Train Loss: {epoch_train_loss:.4f}, Train Accuracy: {epoch_train_acc:.2f}%, '
              f'Val Loss: {epoch_val_loss:.4f}, Val Accuracy: {epoch_val_acc:.2f}%, '
              f'Time: {epoch_time:.2f} sec')

        with open("training_results.txt", "a") as f:
            f.write(f"{epoch+1}, {epoch_train_loss:.4f}, {epoch_train_acc:.2f}, "
                    f"{epoch_val_loss:.4f}, {epoch_val_acc:.2f}, {epoch_time:.2f}\n")

        scheduler.step(epoch_val_loss)
        torch.save(model.state_dict(), f'{model_name}_epoch_{epoch+1}.pth')

    return train_losses, train_accuracies, val_losses, val_accuracies, epoch_times

In [7]:
num_epochs = 30
train_losses, train_accuracies, val_losses, val_accuracies, epoch_times = train_and_validate(
    model, criterion, optimizer, train_loader, val_loader, num_epochs
)

Epoch 1/30 - Training:   0%|          | 0/1811 [00:00<?, ?it/s]c:\Program Files\Python312\Lib\site-packages\torch\nn\functional.py:5560: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)
Validation: 100%|██████████| 453/453 [05:29<00:00,  1.38it/s]


Epoch 1/30, Train Loss: 1.2027, Train Accuracy: 55.23%, Val Loss: 1.0386, Val Accuracy: 61.40%, Time: 2405.71 sec


Validation: 100%|██████████| 453/453 [02:25<00:00,  3.11it/s]


Epoch 2/30, Train Loss: 0.9819, Train Accuracy: 63.86%, Val Loss: 0.9826, Val Accuracy: 63.35%, Time: 1715.11 sec


Validation: 100%|██████████| 453/453 [03:44<00:00,  2.02it/s]


...

Epoch 28/30, Train Loss: 0.1993, Train Accuracy: 92.84%, Val Loss: 0.9642, Val Accuracy: 73.76%, Time: 1549.83 sec


Validation: 100%|██████████| 453/453 [02:25<00:00,  3.11it/s]


Epoch 29/30, Train Loss: 0.2008, Train Accuracy: 92.61%, Val Loss: 0.9572, Val Accuracy: 74.35%, Time: 1550.33 sec


Validation: 100%|██████████| 453/453 [02:41<00:00,  2.80it/s]


Epoch 30/30, Train Loss: 0.1952, Train Accuracy: 92.87%, Val Loss: 0.9787, Val Accuracy: 74.09%, Time: 1542.74 sec
